In [1]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ['CUDA_VISIBLE_DEVICES'] = '5'
import numpy as np
import pandas as pd
import util
df = util.h5_to_df("/bigdata/shared/HepSIM/combo/pythia8_higgs_2_combo.h5")
params = ['Px','Py', 'Pz', 'PT', 'E', 'D0', 'DZ', 'X', 'Y',  'Z', 'T', 'count']
training, target = util.df_to_target(df, output = None, params = params)

Welcome to ROOTaaS 6.06/04
reading file
Generate dictionary
Assigning jet_type...


util.py:371: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  l = np.zeros(length)


In [2]:
def get_sample(training, target, choice):
    target_vals = np.argmax(target, axis = 1)
    ind, = np.where(target_vals == choice)
    chosen_ind = np.random.choice(ind, 30000)
    return training[chosen_ind], target[chosen_ind]

samples = [get_sample(training, target, i) for i in range(5)]
trainings = [i[0] for i in samples]
targets = [i[1] for i in samples]
big_training = np.concatenate(trainings)
big_target = np.concatenate(targets)
big_training, big_target = util.shuffle_together(big_training, big_target)
#big_target = big_target[:, :]
big_target

array([[ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       ..., 
       [ 0.,  0.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.]])

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Masking, Dense, Dropout, Activation, Reshape
from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
m = Sequential()
m.add(Masking(0, input_shape = (util.max_len, len(params))))
m.add(GRU(200, return_sequences = True))
m.add(GRU(5))
m.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
hist6 = m.fit(big_training, big_target, nb_epoch=100, verbose = 1, validation_split=0.2, batch_size = 1000,
              callbacks=[EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min'),
              ModelCheckpoint(filepath='simple3.h5', verbose=0)])

Using TensorFlow backend.


Train on 120000 samples, validate on 30000 samples
Epoch 1/100
120000/120000 [==============================] - 77s - loss: 1.7206 - acc: 0.1561 - val_loss: 1.6307 - val_acc: 0.1258
Epoch 2/100
120000/120000 [==============================] - 76s - loss: 2.5346 - acc: 0.1425 - val_loss: 2.5015 - val_acc: 0.1558
Epoch 3/100
120000/120000 [==============================] - 77s - loss: 2.2369 - acc: 0.1463 - val_loss: 2.1065 - val_acc: 0.1466
Epoch 4/100
120000/120000 [==============================] - 77s - loss: 2.3048 - acc: 0.1353 - val_loss: 2.4987 - val_acc: 0.1453
Epoch 5/100
120000/120000 [==============================] - 77s - loss: 2.4379 - acc: 0.1478 - val_loss: 2.4134 - val_acc: 0.1906
Epoch 6/100
120000/120000 [==============================] - 77s - loss: 2.4145 - acc: 0.2005 - val_loss: 2.3058 - val_acc: 0.1996
Epoch 7/100
120000/120000 [==============================] - 78s - loss: 2.2725 - acc: 0.1902 - val_loss: 2.3288 - val_acc: 0.1892
Epoch 8/100
120000/120000 [=====

I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcublas.so.8.0 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcudnn.so.5 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcufft.so.8.0 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcuda.so.1 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcurand.so.8.0 locally
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE3 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compil

In [ ]:
params = ['Px','Py', 'Pz', 'PT', 'E', 'D0', 'DZ', 'X', 'Y',  'Z', 'count']
training, target = util.df_to_target(df, output = None, params = params)

In [ ]:
%matplotlib inline
import seaborn as sns
sns.set()
ax = sns.violinplot(x = "mom", y = "PT", cut = 0, data = df_sub, scale="area", gridsize = 2000)
ax.set_ylim([0, 20])


In [ ]:
%matplotlib inline
ax = sns.violinplot(x = 'mom', y = 'PT', cut = 0, scale = 'area', data = df.sample(20000), gridsize = 2000)
ax.set_xticklabels(["Other", "H", "H + b"])
ax.set_ylim([0, 20])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
def show_losses( histories ):
    plt.figure(figsize=(10,10))
    #plt.ylim(bottom=0)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Error by Epoch')
    colors=[]
    do_acc=False
    for label,loss in histories:
        color = tuple(np.random.random(3))
        colors.append(color)
        l = label
        vl= label+" validation"
        if 'acc' in loss.history:
            l+=' (acc %2.4f)'% (loss.history['acc'][-1])
            do_acc = True
        if 'val_acc' in loss.history:
            vl+=' (val acc %2.4f)'% (loss.history['val_acc'][-1])
            do_acc = True
        plt.plot(loss.history['loss'], label=l, color=color)
        if 'val_loss' in loss.history:
            plt.plot(loss.history['val_loss'], lw=2, ls='dashed', label=vl, color=color)


    plt.legend()
    plt.yscale('log')
    plt.savefig(l + '-loss.png')
    plt.show()
    if not do_acc: return
    plt.figure(figsize=(10,10))
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    for i,(label,loss) in enumerate(histories):
        color = colors[i]
        if 'acc' in loss.history:
            plt.plot(loss.history['acc'][:-1], lw=2, label=label+" accuracy", color=color)
        if 'val_acc' in loss.history:
            plt.plot(loss.history['val_acc'][:-1], lw=2, ls='dashed', label=label+" validation accuracy", color=color)
    plt.legend(loc='lower right')
    plt.savefig(l + '-acc.png')
    plt.show()

In [ ]:
show_losses([("H + b vs. H vs. not H" , hist5)])

In [ ]:
import matplotlib.pyplot as plt
def stacked_histogram(df, param):
    fig, ax = plt.subplots(1, 1, figsize=(8, 5))
    vals = [df[df.mom == i][param] for i in df.mom.unique()[1:]]
    ax.hist(vals, normed = True, stacked = True, histtype = "stepfilled")#, linewidth = 2)
    #ax.set_yscale('log')
stacked_histogram(df, 'count')

In [ ]:
df